# Mitigating Bias in a Multiclass Classification Model

In this notebook, we will mitigate bias in a multiclass classification model. We will create a pipeline to implement bias mitigation techniques at two different stages: Pre-Processing and Post-Processing. All questions and tasks are bolded and in red.

### 0 - Importing modules and loading the data

We begin by loading the dataset. For this notebook, we will be using the 'USCrime' from the OpenML Repository and load it directly from the holisticai library. The dataset combines socio-economic data from the 1990 US Census, law enforcement data from the 1990 US LEMAS survey, and crime data from the 1995 FBI UCR.

In [1]:
# make sure you have holisticai library installed
!pip install holisticai

In [2]:
# Base Imports
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# import dataset
from holisticai.datasets import  load_us_crime

# import some bias metrics
from holisticai.bias.metrics import multiclass_bias_metrics

# efficacy metrics
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

# import bias mitigation techniques
from holisticai.bias.mitigation import CorrelationRemover
from holisticai.bias.mitigation import FairScoreClassifier
from holisticai.bias.mitigation import MLDebiaser

# import pipeline function
from holisticai.pipeline import Pipeline
from holisticai.utils.transformers.bias import SensitiveGroups


In [3]:
dataset = load_us_crime()
 
df = pd.concat([dataset["data"], dataset["target"]], axis=1)
df_clean = df.iloc[
    :, [i for i, n in enumerate(df.isna().sum(axis=0).T.values) if n < 1000]
]
df_clean = df_clean.dropna()

df_clean

,state,communityname,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,...,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LandArea,PopDens,PctUsePubTrans,LemasPctOfficDrugUn,ViolentCrimesPerPop
0,8.0,Lakewoodcity,1.0,0.19,0.33,0.02,0.90,0.12,0.17,0.34,...,0.12,0.42,0.50,0.51,0.64,0.12,0.26,0.20,0.32,0.20
1,53.0,Tukwilacity,1.0,0.00,0.16,0.12,0.74,0.45,0.07,0.26,...,0.21,0.50,0.34,0.60,0.52,0.02,0.12,0.45,0.00,0.67
2,24.0,Aberdeentown,1.0,0.00,0.42,0.49,0.56,0.17,0.04,0.39,...,0.14,0.49,0.54,0.67,0.56,0.01,0.21,0.02,0.00,0.43
3,34.0,Willingborotownship,1.0,0.04,0.77,1.00,0.08,0.12,0.10,0.51,...,0.19,0.30,0.73,0.64,0.65,0.02,0.39,0.28,0.00,0.12
4,42.0,Bethlehemtownship,1.0,0.01,0.55,0.02,0.95,0.09,0.05,0.38,...,0.11,0.72,0.64,0.61,0.53,0.04,0.09,0.02,0.00,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989,12.0,TempleTerracecity,10.0,0.01,0.40,0.10,0.87,0.12,0.16,0.43,...,0.22,0.28,0.34,0.48,0.39,0.01,0.28,0.05,0.00,0.09
1990,6.0,Seasidecity,10.0,0.05,0.96,0.46,0.28,0.83,0.32,0.69,...,0.53,0.25,0.17,0.10,0.00,0.02,0.37,0.20,0.00,0.45
1991,9.0,Waterburytown,10.0,0.16,0.37,0.25,0.69,0.04,0.25,0.35,...,0.25,0.68,0.61,0.79,0.76,0.08,0.32,0.18,0.91,0.23
1992,25.0,Walthamcity,10.0,0.08,0.51,0.06,0.87,0.22,0.10,0.58,...,0.45,0.64,0.54,0.59,0.52,0.03,0.38,0.33,0.22,0.19


### 1- Pre-processing the data

We are going to prepare the data for the training of a multiclass classfication model. We will use sklearn's logistic regression as our model of choice, and we use its train_test_split function to split our dataset. Note, we do not want to include protected attributes in the training so we will remove any features that contain 'race' and 'age' from the features during training and inference. We also need to create categories for our multiclass problem so we will split the target value into 3 groups using quantiles.

In [4]:
# choose the attribute we want to measure bias with respect to and assign group membership
gs = ['racePctWhite']
group_a = df_clean["racePctWhite"].apply(lambda x: x > 0.5)
group_b = 1 - group_a
xor_groups = group_a ^ group_b

# remove sensitive attributes from the training data
cols = [c for c in df_clean.columns if (not c.startswith('race')) and (not c.startswith('age'))]
df_clean = df_clean[cols].iloc[:, 3:]
df_clean = df_clean[xor_groups]
group_a = group_a[xor_groups]
group_b = group_b[xor_groups]

# standardize data set
scalar = StandardScaler()
df_t = scalar.fit_transform(df_clean)
X = df_t[:, :-1]

# convert the target from a float to 5 categorical groups
def convert_float_to_categorical(target, nb_classes, numeric_classes=True):
    eps = np.finfo(float).eps
    if numeric_classes:
        labels = list(range(nb_classes))
    else:
        labels = [f"Q{c}-Q{c+1}" for c in range(nb_classes)]
    labels_values = np.linspace(0, 1, nb_classes + 1)
    v = np.array(target.quantile(labels_values)).squeeze()
    v[0], v[-1] = v[0] - eps, v[-1] + eps
    y = target.copy()
    for (i, c) in enumerate(labels):
        y[(target.values >= v[i]) & (target.values < v[i + 1])] = c
    return y.astype(np.int32)

nb_classes = 3
y = convert_float_to_categorical(df_clean.iloc[:, -1], nb_classes=nb_classes)


# split the data into training and testing sets
data = X, y, group_a, group_b
datasets = train_test_split(*data, test_size=0.2)
train_data = datasets[::2]
test_data = datasets[1::2]

In [5]:
# efficacy metrics table
def efficacy_metrics(y_pred, y_true):
    acc = accuracy_score(y_true, y_pred)
    ba = balanced_accuracy_score(y_true, y_pred)
    return pd.DataFrame([['Accuracy', acc],['Balanced Accuracy', ba]], columns=['metric', 'value']).set_index('metric')

### 2 - Baseline: training a model and measuring bias

In this section, we will show how to create a pipeline to house a scaler, the model and mitigation techniques. For this particular task we will train a Logistic Regression model from sklearn and then measure bias using the metrics from the multiclass_bias_metrics function.

In [6]:
# create pipeline for model and standardization
pipeline = Pipeline(
    steps=[
        ('scalar', StandardScaler()),
        ("model", LogisticRegression()),
    ]
)

# fit the model to the training data
X, y, group_a, group_b = train_data
pipeline.fit(X, y)

# make a prediction on the testing data
X, y, group_a, group_b = test_data
y_pred = pipeline.predict(X)

# extract the sensitive groups from the testing data
sensgroup = SensitiveGroups()
sens = sensgroup.fit_transform(np.stack([group_a,group_b], axis=1), convert_numeric=True)

# calculate relevant bias metrics for multiclass
df = multiclass_bias_metrics(
    sens,
    y_pred,
    y,
    metric_type='both'
)
y_baseline = y_pred.copy()
df_baseline=df.copy()
df_baseline

,Value,Reference
Metric,,
Max Multiclass Statistical Parity,0.691273,0
Mean Multiclass Statistical Parity,0.691273,0
Max Multiclass Equality of Opportunity,0.478608,0
Max Multiclass Average Odds,0.315913,0
Max Multiclass True Positive Difference,0.409163,0
Mean Multiclass Equality of Opportunity,0.478608,0
Mean Multiclass Average Odds,0.315913,0
Mean Multiclass True Positive Difference,0.409163,0


In [7]:
efficacy_metrics(y_pred, y)

,value
metric,
Accuracy,0.684211
Balanced Accuracy,0.682517


From the summary of bias metrics presented above, we can see that there is bias in the model. Statistical parity (max and mean) are the same in this case as we only consider 2 groups. The value of statistical parity (0.72) is very high. Can we mitigate some bias, while preserving good levels of accuracy/balanced accuracy?

### 3 - Implementing mitigation techniques

In the following sections, we will implement bias mitigation at two different levels: Pre-Processing and Post-Processing.

### 3.1 - Pre-Processing method for Bias Mitigation

As a pre-processing technique, we will be using the [Correlation Remover](https://holisticai.readthedocs.io/en/latest/.generated/holisticai.bias.mitigation.CorrelationRemover.html#holisticai.bias.mitigation.CorrelationRemover). The Correlation Remover applies a linear transformation to the non-sensitive feature columns to remove their correlation with the sensitive feature columns while retaining as much information as possible. Note that the lack of correlation does not imply anything about statistical dependence, for instance information about the protected attributes can still be hidden in pairs of features. Therefore, it is expected this to be most appropriate as a preprocessing step for (generalized) linear models.

In [8]:
# initialize the pipeline 

pipeline = Pipeline(
    steps=[
        ('scalar', StandardScaler()),
        ("bm_preprocessing", CorrelationRemover()),
        ("model", LogisticRegression()),
    ]
)

# prepare training data and parameters
X, y, group_a, group_b = train_data
fit_params = {
    "bm__group_a": group_a, 
    "bm__group_b": group_b
}

# apply steps in pipeline
pipeline.fit(X, y, **fit_params)


# prepare testing data and parameters
X, y, group_a, group_b = test_data
predict_params = {
    "bm__group_a": group_a,
    "bm__group_b": group_b,
}

# make a prediction and generate metrics
y_pred = pipeline.predict(X, **predict_params)

sens = sensgroup.transform(np.stack([group_a,group_b], axis=1), convert_numeric=True)

df = multiclass_bias_metrics(
    sens,
    y_pred,
    y,
    metric_type='both'
)

y_correm  = y_pred.copy()
df_correm =df.copy()
df_correm

,Value,Reference
Metric,,
Max Multiclass Statistical Parity,0.477804,0
Mean Multiclass Statistical Parity,0.477804,0
Max Multiclass Equality of Opportunity,0.375407,0
Max Multiclass Average Odds,0.254802,0
Max Multiclass True Positive Difference,0.362099,0
Mean Multiclass Equality of Opportunity,0.375407,0
Mean Multiclass Average Odds,0.254802,0
Mean Multiclass True Positive Difference,0.362099,0


In [9]:
efficacy_metrics(y_pred, y)

,value
metric,
Accuracy,0.676692
Balanced Accuracy,0.674810


<font color='red'> **Question 1**
- **Has bias increased or decreased after adding the Correlation Remover in the pipeline?**
- **Has efficacy increased or decreased after adding the Correlation Remover in the pipeline?**
<font >

There are improvement with respect to the bias metrics, but not huge improvements. Accuracy decreases but very little.

### 3.2 - Post-Processing Methods for Bias Mitigation

The post-processing technique you will be implementing is the [ML Debiaser](https://holisticai.readthedocs.io/en/latest/.generated/holisticai.bias.mitigation.MLDebiaser.html#holisticai.bias.mitigation.MLDebiaser) (Alabdulmohsin et al, 2021). The algorithm aims to debias predictions w.r.t. the sensitive class in each demographic group. This procedure solves an optimization problem subject to the statistical parity constraint.

<font color='red'> **Task 1**
- **Implement the ML Debiaser post-processing technique (MLDebiaser) using the pipeline method. Generate a summary of bias metrics and efficacy metrics. (Hint: the order of pipeline steps is different than in the pre-processing example.)**
<font >

In [10]:
np.random.seed(10)

pipeline = Pipeline(
    steps=[
        ('scalar', StandardScaler()),
        ("model", LogisticRegression()),
        ("bm_postprocessing", MLDebiaser()),
    ]
)

X, y, group_a, group_b = train_data
fit_params = {
    "bm__group_a": group_a, 
    "bm__group_b": group_b
}

pipeline.fit(X, y, **fit_params)

X, y, group_a, group_b = test_data
predict_params = {
    "bm__group_a": group_a,
    "bm__group_b": group_b,
}
y_pred = pipeline.predict(X, **predict_params)

sens = sensgroup.transform(np.stack([group_a,group_b], axis=1), convert_numeric=True)

df = multiclass_bias_metrics(
    sens,
    y_pred,
    y,
    metric_type='both'
)
y_mldebiaser  = y_pred.copy()
df_mldebiaser = df.copy()
df_mldebiaser

[elapsed time: 00:00:06 | iter:5/5 | primal_residual::4.3141 | dual_residual::0.0317]


,Value,Reference
Metric,,
Max Multiclass Statistical Parity,0.334715,0
Mean Multiclass Statistical Parity,0.334715,0
Max Multiclass Equality of Opportunity,0.255992,0
Max Multiclass Average Odds,0.241725,0
Max Multiclass True Positive Difference,0.213271,0
Mean Multiclass Equality of Opportunity,0.255992,0
Mean Multiclass Average Odds,0.241725,0
Mean Multiclass True Positive Difference,0.213271,0


In [11]:
efficacy_metrics(y_pred, y)

,value
metric,
Accuracy,0.626566
Balanced Accuracy,0.628403


You should get results similar to:
| Metric | Value | Reference |
| ---    | ---   | --- |
Accuracy	|0.65	|1|
Balanced Accuracy	|0.65	|1|

| Metric | Value | Reference |
| ---    | ---   | --- |
Max Multiclass Statistical Parity	|0.34	|0|
Mean Multiclass Statistical Parity	|0.34	|0|
Max Multiclass Equality of Opportunity	|0.22	|0|
Max Multiclass Average Odds	|0.14	|0|
Max Multiclass True Positive Difference	|0.20	|0|
Mean Multiclass Equality of Opportunity		|0.22	|0|
Mean Multiclass Average Odds	|0.14		|0|
Mean Multiclass True Positive Difference		|0.20	|0|


<font color='red'> **Final Question**
- **Assuming we care most about acheiving Statistical Parity, which seems to be the best mitigation technique? Comment on the efficacy tradeoff (if any) to acheive better bias metrics.**
<font >

The ML Debiaser seems to be the best. Although we do suffer some decrease in accuracy to acheive these bias metrics.